In [178]:
%reload_ext autoreload
%autoreload 2

In [404]:
from scholarly import scholarly
import pickle
import numpy as np
import module_scholar
import pandas as pd
import numpy as np
from scholarly import ProxyGenerator
import networkx as nx
from tqdm import tqdm
tqdm.pandas()

pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

with open(r"data/df_authors.pickle", "rb") as file:
    df_authors = pickle.load(file)
with open(r"data/df_publications.pickle", "rb") as file:
    df_publications = pickle.load(file)

In [412]:
PRIMARY_WEIGHT = 3

### Authors

In [444]:
# add author
df_authors = module_scholar.add_author(df_authors, "nd8lQQIAAAAJ")
df_authors.tail(2)

,container_type,filled,scholar_id,source,name,url_picture,affiliation,organization,interests,email_domain,homepage,citedby,citedby5y,hindex,hindex5y,i10index,i10index5y,cites_per_year,coauthors,publications,public_access,primary_org,secondary_orgs,selected,query_name,missing_data,node,relevancy
805,Author,[],5f0v6nkAAAAJ,AuthorSource.SEARCH_AUTHOR_SNIPPETS,Nidhi Sahni,https://scholar.google.com/citations?view_op=m...,The University of Texas MD Anderson Cancer Center,NaN,"[Cancer Biology, Genetics, Gene Regulation]",@mdanderson.org,NaN,5239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,Nidhi Sahni,False,Nidhi Sahni,unknown
806,Author,"[basics, indices, counts, coauthors, publicati...",nd8lQQIAAAAJ,AuthorSource.AUTHOR_PROFILE_PAGE,Christos Faloutsos,https://scholar.googleusercontent.com/citation...,CMU,1.755452e+19,"[Data Mining, Graph Mining, Databases]",@cs.cmu.edu,http://example.edu/~yourname,107392.0,36032.0,147.0,91.0,564.0,388.0,"{1993: 316, 1994: 339, 1995: 496, 1996: 637, 1...","[{'container_type': 'Author', 'filled': [], 's...","[{'container_type': 'Publication', 'source': '...","{'available': 118, 'not_available': 11}",NaN,NaN,True,NaN,NaN,NaN,NaN


In [446]:
# attach organizations to authors
author_name = "Christos Faloutsos"
module_scholar.set_authorCurrentOrg(df_authors, author_name, "carnegie mellon")
module_scholar.add_authorPastOrg(df_authors, author_name, "university of toronto")
df_authors.loc[df_authors["name"]==author_name, "relevancy"] = "relevant"
df_authors[df_authors["name"]==author_name]

,container_type,filled,scholar_id,source,name,url_picture,affiliation,organization,interests,email_domain,homepage,citedby,citedby5y,hindex,hindex5y,i10index,i10index5y,cites_per_year,coauthors,publications,public_access,primary_org,secondary_orgs,selected,query_name,missing_data,node,relevancy
639,Author,[],nd8lQQIAAAAJ,AuthorSource.SEARCH_AUTHOR_SNIPPETS,Christos Faloutsos,https://scholar.google.com/citations?view_op=m...,CMU,NaN,"[Data Mining, Graph Mining, Databases]",@cs.cmu.edu,NaN,107241.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,carnegie mellon,"[carnegie mellon, university of toronto]",False,Christos Faloutsos,False,Christos Faloutsos,relevant
806,Author,"[basics, indices, counts, coauthors, publicati...",nd8lQQIAAAAJ,AuthorSource.AUTHOR_PROFILE_PAGE,Christos Faloutsos,https://scholar.googleusercontent.com/citation...,CMU,1.755452e+19,"[Data Mining, Graph Mining, Databases]",@cs.cmu.edu,http://example.edu/~yourname,107392.0,36032.0,147.0,91.0,564.0,388.0,"{1993: 316, 1994: 339, 1995: 496, 1996: 637, 1...","[{'container_type': 'Author', 'filled': [], 's...","[{'container_type': 'Publication', 'source': '...","{'available': 118, 'not_available': 11}",carnegie mellon,[],True,NaN,NaN,NaN,relevant


In [456]:
# save
with open(r"data/df_authors.pickle", "wb") as file:
    pickle.dump(df_authors, file)

### Publications

In [451]:
# fill publications with authors
df_new_publications = module_scholar.get_newPublications(df_publications, df_authors)
df_publications = pd.concat([df_publications, df_new_publications])
df_publications.loc[:, "pub"].progress_apply(lambda u: u if u["filled"] else scholarly.fill(u, sections="basics"))
df_publications["authors"] = df_publications["pub"].apply(lambda u : frozenset(u.get("bib").get("author").split(" and ")))
df_publications = df_publications.drop_duplicates(subset=["title", "author"]).reset_index(drop=True)
# save
with open("data/df_publications.pickle", "wb") as file:
    pickle.dump(df_publications, file)

100%|██████████| 320/320 [01:47<00:00,  2.99it/s]


### Publications -> Authors

In [452]:
new_authors = module_scholar.get_newAuthorsList(df_publications, df_authors)
new_authors_data = []
count_found = 0
for i, name in enumerate(new_authors):
    ret = scholarly.search_author(name)
    try:
        a = next(ret)
        count_found += 1
        a["query_name"] = name
        new_authors_data.append(a)
    except(StopIteration):
        new_authors_data.append({"query_name":name})
    print(f"Found : {count_found}\t{str(i)}/{len(new_authors)}", end='\r')

In [455]:
df_authors = module_scholar.update_dfAuthors(df_authors, new_authors_data)
df_authors["missing_data"] = df_authors["scholar_id"].isna() & ~(df_authors["query_name"].isna())
df_authors["node"] = np.where(df_authors["name"].isna(), df_authors["query_name"], df_authors["name"])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\leroy\anaconda3\envs\mma-graph\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\leroy\AppData\Local\Temp\ipykernel_14348\1958966833.py", line 1, in <cell line: 1>
    df_authors = module_scholar.update_dfAuthors(df_authors, new_authors_data)
  File "c:\Users\leroy\OneDrive\Bureau\K\Apprentissage\mma-graph\citations_scholar\module_scholar.py", line 162, in update_dfAuthors
    return pd.concat([df_authors, df_new_authors]).reset_index(drop=True).drop_duplicates()
  File "c:\Users\leroy\anaconda3\envs\mma-graph\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\leroy\anaconda3\envs\mma-graph\lib\site-packages\pandas\core\frame.py", line 6122, in drop_duplicates
    duplicated = self.duplicated(subset, keep=keep)
  File "c:\Users\leroy\anaconda3\envs\mma-graph\lib\site-packag

In [457]:
# save
with open(r"data/df_authors.pickle", "wb") as file:
    pickle.dump(df_authors, file)

### Create graph

In [428]:
G = nx.Graph()
module_scholar.add_nodesAuthorsWithAttributes(G, df_authors)
module_scholar.add_nodesPublicationsWithAttributes(G, df_publications)
module_scholar.add_nodesOrganisationWithAttributes(G, df_authors)
module_scholar.add_edgesAuthorsPublications(G, df_publications, df_authors)
module_scholar.add_edgesAuthorsOrganisation(G, df_authors, primary_weight=PRIMARY_WEIGHT)
module_scholar.compute_sizeAuthorsCitations(G, df_authors)
module_scholar.compute_sizePublicationsCitations(G, df_publications)
module_scholar.compute_sizeOrganisationsCitations(G, df_authors, primary_weight=PRIMARY_WEIGHT)

# Export

In [429]:
nx.write_gexf(G, "../data/ns.gexf")

# <>

In [396]:
from random import sample
v = sample(list(G.nodes()), 1)[0]
G.nodes[v]

{'num_citations': 1458, 'bipartite': 'publication', 'size': 1458}

In [341]:
display(df_authors.tail(1))
display(df_publications.head(1))

,container_type,filled,scholar_id,source,name,url_picture,affiliation,organization,interests,email_domain,homepage,citedby,citedby5y,hindex,hindex5y,i10index,i10index5y,cites_per_year,coauthors,publications,public_access,primary_org,secondary_orgs,selected,query_name,missing_data,node
805,Author,[],5f0v6nkAAAAJ,AuthorSource.SEARCH_AUTHOR_SNIPPETS,Nidhi Sahni,https://scholar.google.com/citations?view_op=m...,The University of Texas MD Anderson Cancer Center,NaN,"[Cancer Biology, Genetics, Gene Regulation]",@mdanderson.org,NaN,5239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Nidhi Sahni,False,Nidhi Sahni


,author_pub_id,num_citations,pub_url,url_related_articles,cites_per_year,mandates,pub,title,pub_year,author,id,rank_from_author,n_pub_from_author,authors
0,Q_kKkIUAAAAJ:i2xiXl-TujoC,8255,https://dl.acm.org/doi/abs/10.1145/2939672.293...,/scholar?oi=bibs&hl=en&q=related:SWZQ3L4EwgEJ:...,"{2016: 32, 2017: 288, 2018: 805, 2019: 1423, 2...",[{'agency': 'US National Institutes of Health'...,"{'container_type': 'Publication', 'source': 'P...",node2vec: Scalable feature learning for networks,2016,Q_kKkIUAAAAJ,i2xiXl-TujoC,1.0,500,"(Aditya Grover, Jure Leskovec)"


In [437]:
sorted(list(set().union(*df_authors[~df_authors["primary_org"].isna()]\
                       .apply(
                           lambda u : (set([u["primary_org"]] + u["secondary_orgs"])), axis=1
                           ).tolist())))

['CISPA',
 'carnegie mellon',
 'central european university',
 'chan zuckerberg biohub',
 'citadel llc',
 'harvard',
 'imperial college london',
 'inria',
 'kumo ai',
 'learning planet institute',
 'mcgill university',
 'mila',
 'northeastern university',
 'orange',
 'oxford university',
 'pintereset',
 'princeton',
 'stanford university',
 'technical university of munich',
 'telecom paris',
 'thales',
 'twitter']